# Build a basic chatbot
- Doc: https://langchain-ai.github.io/langgraph/tutorials/get-started/1-build-basic-chatbot/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")

## 2. Create a StateGraph

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# 에이전트의 상태 정의 클래스
# 상태는 워크플로우에서 노드 간 데이터를 전달하는 데 사용
class State(TypedDict):
    # Annotated: 타입 힌트
    # messages는: 리스트, 대화 기록 저장
    # add_messages: 상태 업데이트 시 메시지를 덮어쓰지 않고 추가
    messages: Annotated[list, add_messages]


# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

## 3. Add a node

In [ ]:
# 챗봇 노드 함수
def chatbot(state: State):
    # LLM의 출력(응답)을 messages 리스트에 추가하여 반환
    return {"messages": [llm.invoke(state["messages"])]}


# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

## 4. Add an entry point

In [ ]:
# 워크플로우의 시작점(START)에서 'chatbot' 노드로 이동하는 엣지 추가
graph_builder.add_edge(START, "chatbot")
# 'chatbot' 노드에서 워크플로우의 종료점(END)로 이동하는 엣지 추가
graph_builder.add_edge("chatbot", END)

## 5. Compile the graph

In [ ]:
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성
graph = graph_builder.compile()

## 6. Visualize the graph (optional)

In [ ]:
from IPython.display import Image, display

try:
    # 워크플로우 그래프를 이미지로 시각화하여 출력
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

## 7. Run the chatbot

In [ ]:
def stream_graph_updates(user_input: str):
    # 그래프를 스트리밍 모드로 실행
    events = graph.stream({"messages": [{"role": "user", "content": user_input}]})
    for event in events:
        for value in event.values():
            # 최신 메시지(마지막 메시지)의 내용을 출력
            value["messages"][-1].pretty_print()


while True:
    try:
        user_input = input("User: ")  # 사용자 입력 받기
        if user_input.lower() in ["quit", "exit", "q"]:  # 종료 조건
            print("Goodbye!")
            break
        stream_graph_updates(user_input)  # 워크플로우 실행
    except:
        break